In [ ]:
# !gcloud config set project hca-hin-dev-cur-parallon

In [ ]:
!py -3 -m pip install xlsxwriter

In [ ]:
!py -3 -m pip install pandas

In [ ]:
import os
os.environ["HTTP_PROXY"] = "proxy.nas.medcity.net:80"
os.environ["HTTPS_PROXY"] = "proxy.nas.medcity.net:80"

In [ ]:
import pandas as pd
import xlsxwriter

In [ ]:
csv_files_path_folder = r"InputFiles\csv_files" 
excel_files_path_folder = r"OutputFiles\Convert_excels"
input_file_extension = ".csv"
output_file_extension = ".xlsx"
output_filename = "Load_Discrepancies"

In [ ]:
print(csv_files_path_folder)
print(excel_files_path_folder)

In [ ]:
if not os.path.exists(excel_files_path_folder):
    os.makedirs(excel_files_path_folder)

target_file_path = os.path.join(excel_files_path_folder, output_filename + output_file_extension)

if os.path.exists(target_file_path):
    os.remove(target_file_path)
    write_mode = "w"

file_list = [filename for filename in os.listdir(f"{csv_files_path_folder}") if filename.endswith(input_file_extension)]
with pd.ExcelWriter(target_file_path, mode=write_mode, engine='xlsxwriter') as writer:
    for filename in file_list:
        filename_without_ext = filename.replace(input_file_extension, '')
        source_file_path = os.path.join(csv_files_path_folder, filename)
        read_file = pd.read_csv(source_file_path)
        read_file.columns = ['File Name', 'File Path']
        read_file.to_excel(writer, sheet_name=filename_without_ext, index=False)
        (max_row, max_col) = read_file.shape
        workbook = writer.book
        border_fmt = workbook.add_format({'bottom':2, 'top':2, 'left':2, 'right':2, 'border_color': '#0078D4'})
        worksheet = writer.sheets[filename_without_ext]
        worksheet.conditional_format(xlsxwriter.utility.xl_range(0, 0, max_row, max_col - 1), {'type': 'no_errors', 'format': border_fmt})
        worksheet.autofit()
    writer.save()
    # write_mode = "a"